In [1]:
import sys
sys.executable

'C:\\Users\\wucci_admin\\Anaconda3\\envs\\tfdl02\\python.exe'

In [2]:
# ms-python.python added
import os
'''
try:
	os.chdir(os.path.join(os.getcwd(), '../../../../../../var/folders/lx/703nk8wx7vb585ttwyxk_1lr0000gn/T'))
	print(os.getcwd())
except:
	pass
'''

"\ntry:\n\tos.chdir(os.path.join(os.getcwd(), '../../../../../../var/folders/lx/703nk8wx7vb585ttwyxk_1lr0000gn/T'))\n\tprint(os.getcwd())\nexcept:\n\tpass\n"

In [3]:
from IPython import get_ipython

 # Training Model
 Keras on Tensorflow


In [4]:
# import os, glob, shutil
import os, sys
import cv2
import numpy as np
import uuid
import tensorflow as tf
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from imutils import paths
import itertools
import json

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from core.imageprep import random_crop, crop_generator, random_crop_batch
from core.models import UNet

from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from datetime import datetime

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

In [6]:
%load_ext tensorboard

 ## Load Training Dataset

In [7]:
# load image
print("Load Images...")
# on mac
# path = "/Volumes/LaCie_DataStorage/PerlmutterData/"

# on Window PC 
path = os.path.join('D:', 'PerlmutterData')
imginput = os.path.join('dl_seg_project_raw', 'data_crop', '2019_12_06_17_06',)
imgpath = os.path.join(path, imginput)

print(imgpath)

img_dir = os.path.join(imgpath, 'images')
label_dir = os.path.join(imgpath, 'labels')
print(img_dir)
print(label_dir)

Load Images...
D:PerlmutterData\dl_seg_project_raw\data_crop\2019_12_06_17_06
D:PerlmutterData\dl_seg_project_raw\data_crop\2019_12_06_17_06\images
D:PerlmutterData\dl_seg_project_raw\data_crop\2019_12_06_17_06\labels


 Print the first file name.

In [8]:
imgpath_all = list(paths.list_images(imgpath))
print(imgpath_all[0])

D:PerlmutterData\dl_seg_project_raw\data_crop\2019_12_06_17_06\images\autophagosome\0001.tif


 ## Create Image Datagenerator
 1. create only one datagen
 2. specify valiation split in datagen argument
 3. add split data when calling `datagen.flow_from_directory`

In [9]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
seed = 100
batch_size = 16
epoch = 500
validation_steps = 20
validation_split = 0.1
training_sample_size = len(imgpath_all)
IMG_HEIGHT = 256
IMG_WIDTH = 256
classes = ['cell_membrane', 'nucleus', 'autophagosome']
inputclass = [classes[0]]
learning_rate = 1e-5
loss = "binary_crossentropy"
metrics = ['accuracy']

In [10]:
# create arguments for data generator
data_gen_img_args = dict(
                # featurewise_center = True,
                # featurewise_std_normalization = True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                rescale=1.0/255.0,
                )

data_gen_label_args = dict(
                # featurewise_center=True,
                # featurewise_std_normalization=True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                # rescale=1.0/255.0,
                rescale=1.0/255.0,
                )

# create parameter
pars = dict(
                timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M"),
                seed = seed,
                batch_size = batch_size,
                epoch = epoch,
                validation_steps = validation_steps,
                validation_split = validation_split,
                training_sample_size = training_sample_size,
                IMG_HEIGHT = IMG_HEIGHT,
                IMG_WIDTH = IMG_WIDTH,
                # training class
                classes = classes,
                inputclass = inputclass,
                # add datagen args
                data_gen_img_args = data_gen_img_args,
                data_gen_label_args = data_gen_label_args,
                # hyper parameter
                loss = loss,
                metrics = metrics,
                learning_rate = learning_rate,
                )

# save parameter
if not 'pars' in os.listdir(path):
    os.mkdir(os.path.join(path, 'pars'))
    
jsonpath = os.path.join(path, 'pars')

with open(os.path.join(jsonpath, 'pars_' + timestamp + '.json'), 'w') as outfile:
    json.dump(pars, outfile, indent=4)

In [11]:
# create generator
image_datagen = ImageDataGenerator(**data_gen_img_args)
label_datagen = ImageDataGenerator(**data_gen_label_args)

In [12]:
# load images into generator
train_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training', # <- define subset as 'training'
    seed=seed)

train_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training',
    seed=seed)

valid_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation', # <- define subset as 'validation'
    seed=seed)

valid_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation',
    seed=seed)



Found 8100 images belonging to 1 classes.
Found 8100 images belonging to 1 classes.
Found 900 images belonging to 1 classes.
Found 900 images belonging to 1 classes.


In [13]:

# merge image and label generator
def combine_generator(gen1, gen2):
    while True:
        yield(gen1.next(), gen2.next()) 
train_generator = combine_generator(train_image_generator, train_label_generator)
valid_generator = combine_generator(valid_image_generator, valid_label_generator)


In [14]:
'''
train_generator = zip(train_image_generator, train_label_generator)
valid_generator = zip(valid_image_generator, valid_label_generator)
'''

'\ntrain_generator = zip(train_image_generator, train_label_generator)\nvalid_generator = zip(valid_image_generator, valid_label_generator)\n'

 ## Training

In [15]:
print("Start training...")

Start training...


 ### Define checkpointer

In [16]:
# checkpointer
# check if the output folder exist. If not, create a folder
if not 'model' in os.listdir(path):
    os.mkdir(os.path.join(path, 'model'))

if not 'logs' in os.listdir(path):
    os.mkdir(os.path.join(path, 'logs'))
    os.mkdir(os.path.join(path, 'logs', 'fit'))
    
# set up the checkpointer
# save model
modelfilename = 'model_' + timestamp + '.h5'
modelfilepath = os.path.join(path, 'model', modelfilename)

checkpointer = ModelCheckpoint(modelfilepath, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

log_dir=os.path.join(path, 'logs', 'fit', datetime.now().strftime("%Y%m%d-%H%M%S"))

# add tensorboard
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

 ### Setup the model

In [17]:
# calculate steps_per_epoch
steps_per_epoch = training_sample_size * (1-validation_split) // batch_size
print("Steps per epoch: {}".format(steps_per_epoch))

Steps per epoch: 3043.0


In [ ]:
# prepare the model
unetmodel = UNet(shape = [IMG_HEIGHT, IMG_WIDTH], 
                 lr = learning_rate, 
                 loss = loss,
                 metrics = metrics,
                )

# train the model
unetmodel.fit_generator(
                    generator=train_generator, 
                    validation_data=valid_generator,
                    validation_steps=validation_steps,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epoch, 
                    verbose=1, 
                    # callbacks=[checkpointer], 
                    callbacks=[checkpointer, tensorboard_callback]
                    )

conv1 shape: (None, 256, 256, 64)
conv1 shape: (None, 256, 256, 64)
pool1 shape: (None, 128, 128, 64)
conv2 shape: (None, 128, 128, 128)
conv2 shape: (None, 128, 128, 128)
pool2 shape: (None, 64, 64, 128)
conv3 shape: (None, 64, 64, 256)
conv3 shape: (None, 64, 64, 256)
pool3 shape: (None, 32, 32, 256)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]        

Epoch 1/500
3042/3043 [============================>.] - ETA: 0s - loss: 0.0841 - accuracy: 0.9686
Epoch 00001: val_loss improved from inf to 0.01990, saving model to D:PerlmutterData\model\model_2019_12_06_17_14.h5
3043/3043 [==============================] - 2980s 979ms/step - loss: 0.0841 - accuracy: 0.9686 - val_loss: 0.0199 - val_accuracy: 0.9933
Epoch 2/500
3042/3043 [============================>.] - ETA: 0s - loss: 0.0346 - accuracy: 0.9872
Epoch 00002: val_loss did not improve from 0.01990
3043/3043 [==============================] - 2980s 979ms/step - loss: 0.0346 - accuracy: 0.9872 - val_loss: 0.0447 - val_accuracy: 0.9866
Epoch 3/500
3042/3043 [============================>.] - ETA: 0s - loss: 0.0269 - accuracy: 0.9898
Epoch 00003: val_loss improved from 0.01990 to 0.01989, saving model to D:PerlmutterData\model\model_2019_12_06_17_14.h5
3043/3043 [==============================] - 2983s 980ms/step - loss: 0.0269 - accuracy: 0.9898 - val_loss: 0.0199 - val_accuracy: 0.9915


In [ ]:
%tensorboard --logdir D:/PerlmetterData/logs/fit

In [ ]:
!nvidia-smi